<a href="https://colab.research.google.com/github/marilynle/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/Marilyn_Landim_Esko_LS_DS_131_Statistics_Probability_and_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>

<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Statistics, Probability and Inference

## Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

Load and clean the data (or determine the best method to drop observations when running tests)
Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)
Note that this data will involve 2 sample t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

Refactor your code into functions so it's easy to rerun with arbitrary variables
Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)


In [0]:
import pandas as pd
import numpy as np
import scipy.stats


In [2]:
# Checking the raw data first
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
  

republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y
democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
democrat,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y
republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?
republican,n,y,n,y,y,n,n,n,n,n,?,?,y,y,n,n
republican,n,y,n,y,y,y,n,n,n,n,y,?,y,y,?,?
democrat,n,y,y,n,n,n,y,y,y,n,n,n,y,n,?,?
democrat,y,y,y,n,n,y,y,y,?,y,y,?,n,n,y,?
republican,n,y,n,y,y,y,n,n,n,n,n,y,?,?,n,?
republican,n,y,n,y,y,y,n,n,n,y,n,y,y,?,n,?
democrat,y,n,y,n,n,y,n,y,?,y,y,y,?,n,n,y
democrat,y,?,y,n,n,n,y,y,y,n,n,n,y,n,y,y
republican,n,y,n,y,y,y,n,n,n,n,n,?,y,y,n,n
democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
democrat,y,y,y,n,n,?,y,y,n,n,y,n,n,n,y,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,?,?,y,y
democrat,y,?,y,n,n,n,y,y,y,n,n,?,n,n,y,y
democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,

In [3]:
# Checking Attribute Information
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names
  

1. Title: 1984 United States Congressional Voting Records Database

2. Source Information:
    (a) Source:  Congressional Quarterly Almanac, 98th Congress, 
                 2nd session 1984, Volume XL: Congressional Quarterly Inc. 
                 Washington, D.C., 1985.
    (b) Donor: Jeff Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
    (c) Date: 27 April 1987 

3. Past Usage
   - Publications
     1. Schlimmer, J. C. (1987).  Concept acquisition through 
        representational adjustment.  Doctoral dissertation, Department of 
        Information and Computer Science, University of California, Irvine, CA.
        -- Results: about 90%-95% accuracy appears to be STAGGER's asymptote
     - Predicted attribute: party affiliation (2 classes)

4. Relevant Information:
      This data set includes votes for each of the U.S. House of
      Representatives Congressmen on the 16 key votes identified by the
      CQA.  The CQA lists nine different types of votes: voted for, paired
      

In [4]:
 # Loading the data
  !wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
    

--2019-10-03 16:33:30--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.1s    

2019-10-03 16:33:31 (124 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [5]:
# Adding a header row and cleaning the data
df = pd.read_csv('house-votes-84.data',
               header=None,
               names=['class_name','handicapped_infants','water_project',
                        'budget_resolution','physician_fee_freeze', 'el_salvador_aid',
                        'religious_groups_in_schools','anti_satellite_test_ban',
                        'aid_to_nicaraguan_contras','mx_missile','immigration',
                        'synfuels_corporation_cutback', 'education_spending', 'superfund_right_to_sue','crime','duty_free_exports',
                        'export_administration_act_south_africa'])
df = df.replace({'?':np.NaN,'n':0,'y':1})
df.sample(10)


,class_name,handicapped_infants,water_project,budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
46,democrat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
58,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
126,republican,0.0,NaN,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
170,democrat,1.0,0.0,1.0,0.0,0.0,0.0,NaN,1.0,1.0,NaN,0.0,0.0,0.0,0.0,1.0,NaN
184,democrat,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0
265,democrat,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
415,democrat,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,1.0,0.0,0.0,0.0,0.0,0.0,1.0
386,democrat,0.0,NaN,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN
293,democrat,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
270,democrat,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [6]:
# Finding the number of NaN in each column
df.isnull().sum()


class_name                                  0
handicapped_infants                        12
water_project                              48
budget_resolution                          11
physician_fee_freeze                       11
el_salvador_aid                            15
religious_groups_in_schools                11
anti_satellite_test_ban                    14
aid_to_nicaraguan_contras                  15
mx_missile                                 22
immigration                                 7
synfuels_corporation_cutback               21
education_spending                         31
superfund_right_to_sue                     25
crime                                      17
duty_free_exports                          28
export_administration_act_south_africa    104
dtype: int64

In [7]:
df.describe()


,handicapped_infants,water_project,budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
count,423.000000,387.000000,424.000000,424.000000,420.000000,424.000000,421.000000,420.00000,413.000000,428.000000,414.000000,404.000000,410.000000,418.000000,407.000000,331.000000
mean,0.442080,0.503876,0.596698,0.417453,0.504762,0.641509,0.567696,0.57619,0.501211,0.504673,0.362319,0.423267,0.509756,0.593301,0.427518,0.812689
std,0.497222,0.500632,0.491140,0.493721,0.500574,0.480124,0.495985,0.49475,0.500605,0.500563,0.481252,0.494690,0.500516,0.491806,0.495327,0.390752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
# Using a two-sample hypothesis testing

In [0]:
# The samples
df_democrat = df[df['class_name'] == 'democrat']
df_republican = df[df['class_name'] == 'republican']


In [10]:
df_democrat.shape


(267, 17)

In [11]:
df_republican.shape


(168, 17)

In [0]:
issues =['handicapped_infants','water_project',
                        'budget_resolution','physician_fee_freeze', 'el_salvador_aid',
                        'religious_groups_in_schools','anti_satellite_test_ban',
                        'aid_to_nicaraguan_contras','mx_missile','immigration',
                        'synfuels_corporation_cutback', 'education_spending', 'superfund_right_to_sue','crime','duty_free_exports',
                        'export_administration_act_south_africa']


In [13]:
# The mean of democrats that support each issue (that voted yes for the issue)
Democrats = df_democrat[issues].mean()
Democrats


handicapped_infants                       0.604651
water_project                             0.502092
budget_resolution                         0.888462
physician_fee_freeze                      0.054054
el_salvador_aid                           0.215686
religious_groups_in_schools               0.476744
anti_satellite_test_ban                   0.772201
aid_to_nicaraguan_contras                 0.828897
mx_missile                                0.758065
immigration                               0.471483
synfuels_corporation_cutback              0.505882
education_spending                        0.144578
superfund_right_to_sue                    0.289683
crime                                     0.350195
duty_free_exports                         0.637450
export_administration_act_south_africa    0.935135
dtype: float64

In [14]:
# The mean of republican that support each issue (that voted yes for the issue)
Republicans = df_republican[issues].mean()
Republicans


handicapped_infants                       0.187879
water_project                             0.506757
budget_resolution                         0.134146
physician_fee_freeze                      0.987879
el_salvador_aid                           0.951515
religious_groups_in_schools               0.897590
anti_satellite_test_ban                   0.240741
aid_to_nicaraguan_contras                 0.152866
mx_missile                                0.115152
immigration                               0.557576
synfuels_corporation_cutback              0.132075
education_spending                        0.870968
superfund_right_to_sue                    0.860759
crime                                     0.981366
duty_free_exports                         0.089744
export_administration_act_south_africa    0.657534
dtype: float64

In [0]:
# Stretch goals:
# Refactor your code into functions so it's easy to rerun with arbitrary variables 

In [0]:
def hypothesis_testing (a,b): # Using hypothesis testing, find an issue
  
  Pvalue = scipy.stats.ttest_ind(a,b,nan_policy='omit',equal_var=False).pvalue
  print(f'Pvalue is: {Pvalue:.4f}.'.format(Pvalue))      
  
  if a.mean() > b.mean() and Pvalue < 0.01:  # that democrats support more than republicans with p < 0.01
    print("This issue democrats support more than the republicans")
    
  elif a.mean() < b.mean() and Pvalue < 0.01:   # that republicans support more than democrats with p < 0.01 
    print("This issue republicans support more than the democrats")
    
  else:   # where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)
    print("Both republicans and democrats support this issue")
    
 

In [17]:
for issue in issues:   
    print("") 
    print(issue)
    hypothesis_testing(df_democrat[issue], df_republican[issue]) # calling the function for each issue


handicapped_infants
Pvalue is: 0.0000.
This issue democrats support more than the republicans

water_project
Pvalue is: 0.9292.
Both republicans and democrats support this issue

budget_resolution
Pvalue is: 0.0000.
This issue democrats support more than the republicans

physician_fee_freeze
Pvalue is: 0.0000.
This issue republicans support more than the democrats

el_salvador_aid
Pvalue is: 0.0000.
This issue republicans support more than the democrats

religious_groups_in_schools
Pvalue is: 0.0000.
This issue republicans support more than the democrats

anti_satellite_test_ban
Pvalue is: 0.0000.
This issue democrats support more than the republicans

aid_to_nicaraguan_contras
Pvalue is: 0.0000.
This issue democrats support more than the republicans

mx_missile
Pvalue is: 0.0000.
This issue democrats support more than the republicans

immigration
Pvalue is: 0.0832.
Both republicans and democrats support this issue

synfuels_corporation_cutback
Pvalue is: 0.0000.
This issue democrats 